In [1]:
import sys
sys.path.append('..')
# %%
import os
from matplotlib import pyplot as plt
import numpy as np
import h5py
from sklearn.model_selection import train_test_split

import torch
from utilities import WaveformDataset

In [2]:
torch.cuda.is_available()

True

In [3]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [4]:
cuda = torch.device('cuda')

In [5]:
try_gpu()

device(type='cuda', index=0)

In [6]:
# %% Read the pre-processed datasets
model_datasets = '../training_datasets/training_datasets_STEAD_waveform.hdf5'
with h5py.File(model_datasets, 'r') as f:
    X_train = f['X_train'][:]
    Y_train = f['Y_train'][:]

# 3. split to training (60%), validation (20%) and test (20%)
train_size = 0.6
rand_seed1 = 13
rand_seed2 = 20
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, train_size=0.1, random_state=rand_seed1)
X_validate, X_test, Y_validate, Y_test = train_test_split(X_test, Y_test, test_size=0.9, random_state=rand_seed2)

In [7]:
# Convert to the dataset class for Pytorch (here simply load all the data,
# but for the sake of memory, can also use WaveformDataset_h5)
training_data = WaveformDataset(X_train, Y_train)
validate_data = WaveformDataset(X_validate, Y_validate)

In [10]:
# load mini-batch (testing)
from torch.utils.data import DataLoader
from autoencoder_1D_models_torch import Autoencoder_Conv1D_deep

def training_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 20 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss= 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Test Avg loss: {test_loss:>8f}\n")

In [12]:
batch_size, epochs, lr = 128, 200, 1e-3
model = Autoencoder_Conv1D_deep().to(device=try_gpu())
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
validate_dataloader = DataLoader(validate_data, batch_size=batch_size, shuffle=True)

for t in range(epochs):
    print(f"Epoch {t + 1}\n ===========================")
    training_loop(train_dataloader, model, loss_fn, optimizer, try_gpu())
    test_loop(validate_dataloader, model, loss_fn, try_gpu())
print("Done!")

Epoch 1
loss: 2.000715 [    0/10000]
loss: 1.287735 [ 2560/10000]
loss: 0.990403 [ 5120/10000]
loss: 0.848007 [ 7680/10000]
Test Avg loss: 0.761452

Epoch 2
loss: 0.751938 [    0/10000]
loss: 0.678050 [ 2560/10000]
loss: 0.573800 [ 5120/10000]
loss: 0.500830 [ 7680/10000]
Test Avg loss: 0.449007

Epoch 3
loss: 0.452714 [    0/10000]
loss: 0.393113 [ 2560/10000]
loss: 0.348840 [ 5120/10000]
loss: 0.336185 [ 7680/10000]
Test Avg loss: 0.313086

Epoch 4
loss: 0.321363 [    0/10000]
loss: 0.279932 [ 2560/10000]
loss: 0.256149 [ 5120/10000]
loss: 0.210433 [ 7680/10000]
Test Avg loss: 0.186103

Epoch 5
loss: 0.195432 [    0/10000]
loss: 0.151620 [ 2560/10000]
loss: 0.150982 [ 5120/10000]
loss: 0.131150 [ 7680/10000]
Test Avg loss: 0.113603

Epoch 6
loss: 0.112633 [    0/10000]
loss: 0.105765 [ 2560/10000]
loss: 0.078263 [ 5120/10000]
loss: 0.074354 [ 7680/10000]
Test Avg loss: 0.067429

Epoch 7
loss: 0.071728 [    0/10000]
loss: 0.059278 [ 2560/10000]
loss: 0.050724 [ 5120/10000]
loss: 0.050

Test Avg loss: 0.007190

Epoch 47
loss: 0.006711 [    0/10000]
loss: 0.005609 [ 2560/10000]
loss: 0.009005 [ 5120/10000]
loss: 0.006028 [ 7680/10000]
Test Avg loss: 0.006612

Epoch 48
loss: 0.008152 [    0/10000]
loss: 0.006465 [ 2560/10000]
loss: 0.006727 [ 5120/10000]
loss: 0.005948 [ 7680/10000]
Test Avg loss: 0.007771

Epoch 49
loss: 0.006667 [    0/10000]
loss: 0.007474 [ 2560/10000]
loss: 0.007459 [ 5120/10000]
loss: 0.006075 [ 7680/10000]
Test Avg loss: 0.010729

Epoch 50
loss: 0.008844 [    0/10000]
loss: 0.009420 [ 2560/10000]
loss: 0.006954 [ 5120/10000]
loss: 0.006885 [ 7680/10000]
Test Avg loss: 0.007613

Epoch 51
loss: 0.007585 [    0/10000]
loss: 0.008125 [ 2560/10000]
loss: 0.006046 [ 5120/10000]
loss: 0.006239 [ 7680/10000]
Test Avg loss: 0.006356

Epoch 52
loss: 0.005661 [    0/10000]
loss: 0.009254 [ 2560/10000]
loss: 0.008121 [ 5120/10000]
loss: 0.007956 [ 7680/10000]
Test Avg loss: 0.007979

Epoch 53
loss: 0.009786 [    0/10000]
loss: 0.008434 [ 2560/10000]
loss: 0.

loss: 0.004710 [ 7680/10000]
Test Avg loss: 0.004286

Epoch 93
loss: 0.004161 [    0/10000]
loss: 0.005117 [ 2560/10000]
loss: 0.004403 [ 5120/10000]
loss: 0.004042 [ 7680/10000]
Test Avg loss: 0.004984

Epoch 94
loss: 0.004678 [    0/10000]
loss: 0.004518 [ 2560/10000]
loss: 0.006797 [ 5120/10000]
loss: 0.004364 [ 7680/10000]
Test Avg loss: 0.004311

Epoch 95
loss: 0.004370 [    0/10000]
loss: 0.003929 [ 2560/10000]
loss: 0.004590 [ 5120/10000]
loss: 0.003588 [ 7680/10000]
Test Avg loss: 0.005163

Epoch 96
loss: 0.005562 [    0/10000]
loss: 0.004037 [ 2560/10000]
loss: 0.003335 [ 5120/10000]
loss: 0.005528 [ 7680/10000]
Test Avg loss: 0.005168

Epoch 97
loss: 0.004505 [    0/10000]
loss: 0.004449 [ 2560/10000]
loss: 0.003460 [ 5120/10000]
loss: 0.002693 [ 7680/10000]
Test Avg loss: 0.008207

Epoch 98
loss: 0.006833 [    0/10000]
loss: 0.004690 [ 2560/10000]
loss: 0.004817 [ 5120/10000]
loss: 0.005502 [ 7680/10000]
Test Avg loss: 0.004405

Epoch 99
loss: 0.005151 [    0/10000]
loss: 0.

loss: 0.004069 [ 2560/10000]
loss: 0.002347 [ 5120/10000]
loss: 0.003106 [ 7680/10000]
Test Avg loss: 0.004346

Epoch 139
loss: 0.004464 [    0/10000]
loss: 0.003969 [ 2560/10000]
loss: 0.001904 [ 5120/10000]
loss: 0.003298 [ 7680/10000]
Test Avg loss: 0.003836

Epoch 140
loss: 0.003841 [    0/10000]
loss: 0.002085 [ 2560/10000]
loss: 0.002724 [ 5120/10000]
loss: 0.002976 [ 7680/10000]
Test Avg loss: 0.002959

Epoch 141
loss: 0.002442 [    0/10000]
loss: 0.002252 [ 2560/10000]
loss: 0.002441 [ 5120/10000]
loss: 0.002963 [ 7680/10000]
Test Avg loss: 0.006476

Epoch 142
loss: 0.006460 [    0/10000]
loss: 0.003846 [ 2560/10000]
loss: 0.002822 [ 5120/10000]
loss: 0.002707 [ 7680/10000]
Test Avg loss: 0.003722

Epoch 143
loss: 0.003243 [    0/10000]
loss: 0.002310 [ 2560/10000]
loss: 0.002930 [ 5120/10000]
loss: 0.003098 [ 7680/10000]
Test Avg loss: 0.006885

Epoch 144
loss: 0.006092 [    0/10000]
loss: 0.005001 [ 2560/10000]
loss: 0.002821 [ 5120/10000]
loss: 0.003060 [ 7680/10000]
Test Av

loss: 0.001886 [ 2560/10000]
loss: 0.004112 [ 5120/10000]
loss: 0.003018 [ 7680/10000]
Test Avg loss: 0.002841

Epoch 185
loss: 0.001789 [    0/10000]
loss: 0.001771 [ 2560/10000]
loss: 0.002687 [ 5120/10000]
loss: 0.002786 [ 7680/10000]
Test Avg loss: 0.002624

Epoch 186
loss: 0.002461 [    0/10000]
loss: 0.001969 [ 2560/10000]
loss: 0.002545 [ 5120/10000]
loss: 0.003351 [ 7680/10000]
Test Avg loss: 0.002938

Epoch 187
loss: 0.002342 [    0/10000]
loss: 0.002147 [ 2560/10000]
loss: 0.002417 [ 5120/10000]
loss: 0.002510 [ 7680/10000]
Test Avg loss: 0.002395

Epoch 188
loss: 0.001634 [    0/10000]
loss: 0.002440 [ 2560/10000]
loss: 0.002676 [ 5120/10000]
loss: 0.001683 [ 7680/10000]
Test Avg loss: 0.002669

Epoch 189
loss: 0.002251 [    0/10000]
loss: 0.002136 [ 2560/10000]
loss: 0.002772 [ 5120/10000]
loss: 0.001673 [ 7680/10000]
Test Avg loss: 0.002599

Epoch 190
loss: 0.002793 [    0/10000]
loss: 0.002167 [ 2560/10000]
loss: 0.002067 [ 5120/10000]
loss: 0.002162 [ 7680/10000]
Test Av

In [19]:
model = Autoencoder_Conv1D_deep().to(device=try_gpu())

In [21]:
next(model.parameters()).is_cuda

True